# Modeling: Softmax Regression, (Logistic regression)

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier



In [2]:
data = pd.read_csv('ObesityDataset.csv')
print(data.head())

   Gender   Age family_history_with_overweight FAVC  FCVC  NCP       CAEC  \
0  Female  21.0                            yes   no   2.0  3.0  Sometimes   
1  Female  21.0                            yes   no   3.0  3.0  Sometimes   
2    Male  23.0                            yes   no   2.0  3.0  Sometimes   
3    Male  27.0                             no   no   3.0  3.0  Sometimes   
4    Male  22.0                             no   no   2.0  1.0  Sometimes   

  SMOKE  CH2O  SCC  FAF  TUE        CALC                 MTRANS  \
0    no   2.0   no  0.0  1.0          no  Public_Transportation   
1   yes   3.0  yes  3.0  0.0   Sometimes  Public_Transportation   
2    no   2.0   no  2.0  1.0  Frequently  Public_Transportation   
3    no   2.0   no  2.0  0.0  Frequently                Walking   
4    no   2.0   no  0.0  0.0   Sometimes  Public_Transportation   

            NObeyesdad  
0        Normal_Weight  
1        Normal_Weight  
2        Normal_Weight  
3   Overweight_Level_I  
4  Overwe

# 1.1 Data preparation for modeling

## Train-test split

In [3]:
CSV_PATH = "ObesityDataset.csv"  
data = pd.read_csv(CSV_PATH)

print(data)

      Gender        Age family_history_with_overweight FAVC  FCVC  NCP  \
0     Female  21.000000                            yes   no   2.0  3.0   
1     Female  21.000000                            yes   no   3.0  3.0   
2       Male  23.000000                            yes   no   2.0  3.0   
3       Male  27.000000                             no   no   3.0  3.0   
4       Male  22.000000                             no   no   2.0  1.0   
...      ...        ...                            ...  ...   ...  ...   
2106  Female  20.976842                            yes  yes   3.0  3.0   
2107  Female  21.982942                            yes  yes   3.0  3.0   
2108  Female  22.524036                            yes  yes   3.0  3.0   
2109  Female  24.361936                            yes  yes   3.0  3.0   
2110  Female  23.664709                            yes  yes   3.0  3.0   

           CAEC SMOKE      CH2O  SCC       FAF       TUE        CALC  \
0     Sometimes    no  2.000000   no  0

In [4]:
TARGET_COL = "NObeyesdad" 

X = data.drop(columns=[TARGET_COL])
y = data[TARGET_COL]


X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42,
)

In [5]:
print(f"Size of data original{X.shape}")
print(f"Size of label original{y.shape}")

print(f"Size of data train{X_train.shape}")
print(f"Size of label train{y_train.shape}")
print(f"Size of data test{X_test.shape}")
print(f"Size of label test{y_test.shape}")


Size of data original(2111, 14)
Size of label original(2111,)
Size of data train(1688, 14)
Size of label train(1688,)
Size of data test(423, 14)
Size of label test(423,)


In [6]:
import numpy as np

def random_state(seed=42):
    np.random.seed(seed)
    
random_state(42)


## Preprocessing:

In [7]:
num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
print(f"Num_cols {num_cols}")
cat_cols = [c for c in X_train.columns if c not in num_cols]
print(f"Cat_cols {cat_cols}")

numeric_tf = Pipeline(steps=[
    ("scaler", StandardScaler()),
])

categorical_tf = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_tf, num_cols),
        ("cat", categorical_tf, cat_cols),
    ],
    remainder="drop",
)

Num_cols ['Age', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
Cat_cols ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']


In [8]:
print(f"preprocess{preprocess}")

preprocessColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['Age', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']),
                                ('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Gender', 'family_history_with_overweight',
                                  'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC',
                                  'MTRANS'])])


In [9]:
print(X_train)

      Gender        Age family_history_with_overweight FAVC      FCVC  \
459     Male  19.000000                            yes  yes  2.000000   
426     Male  22.000000                             no   no  2.000000   
326     Male  18.000000                            yes  yes  3.000000   
971     Male  19.506389                            yes  yes  2.793561   
892   Female  17.085250                             no  yes  1.972545   
...      ...        ...                            ...  ...       ...   
90    Female  25.000000                             no   no  3.000000   
1439  Female  40.654155                            yes  yes  2.000000   
609     Male  19.979810                            yes  yes  2.000000   
1589    Male  38.523646                            yes  yes  2.177896   
478     Male  19.000000                             no  yes  2.000000   

           NCP        CAEC SMOKE      CH2O  SCC       FAF       TUE  \
459   3.000000  Frequently    no  3.000000   no  1.0

## 1.2 Build classification models

+ Sử dụng model:
    + Logistic Regression: là một phương pháp thống kê được sử dụng để mô hình hóa và dự đoán xác suất xảy ra của một biến phụ thuộc nhị phân (có hai lựa chọn: 0 hoặc 1) dựa trên các biến độc lập.
    
    + Decision Tree: Trong lý thuyết quyết định, một cây quyết định là một đồ thị của các quyết định và các hậu quả có thể của nó. Cây quyết định được sử dụng để xây dựng một kế hoạch nhằm đạt được mục tiêu mong muốn. Các cây quyết định được dùng để hỗ trợ quá trình ra quyết định. Cây quyết định là một dạng đặc biệt của cấu trúc cây.

    + Random Forest: Rừng ngẫu nhiên hoặc rừng quyết định ngẫu nhiên là một phương pháp học tập tổng hợp để phân loại, hồi quy và các nhiệm vụ khác hoạt động bằng cách tạo ra vô số cây quyết định trong quá trình đào tạo. Đối với các nhiệm vụ phân loại, đầu ra của rừng ngẫu nhiên là lớp được hầu hết các cây chọn.

    + K-Nearest Neighbor: Trong thống kê, giải thuật k hàng xóm gần nhất là một phương pháp thống kê phi tham số được đề xuất bởi Thomas M. Cover để sử dụng cho phân loại bằng thống kê và phân tích hồi quy. Cụm từ hàng xóm có thể hiểu là láng giềng hoặc lân cận.

In [10]:
random_state(42)
models = {
    "logistic_regresion": LogisticRegression(max_iter=2000, random_state=42),
    "decision_tree": DecisionTreeClassifier(random_state=42),
    "random_forest": RandomForestClassifier(random_state=42, n_estimators=200, max_depth=20, min_samples_leaf=1),
    "knn": KNeighborsClassifier(n_neighbors=3),
}

In [11]:
print(models.items())

dict_items([('logistic_regresion', LogisticRegression(max_iter=2000, random_state=42)), ('decision_tree', DecisionTreeClassifier(random_state=42)), ('random_forest', RandomForestClassifier(max_depth=20, n_estimators=200, random_state=42)), ('knn', KNeighborsClassifier(n_neighbors=3))])


## 1.3 Model outputs for evaluation

In [12]:
def evaluate(pipe: Pipeline, X_test, y_test):
    y_pred = pipe.predict(X_test)
    #print(y_pred)
    #print(y_test)
    if hasattr(pipe, "predict_proba"):
        y_proba = pipe.predict_proba(X_test)
        # print(f"Test acc {y_proba}")
        #print(y_proba)
        auc = roc_auc_score(y_test, y_proba, multi_class="ovr", average="macro")
    else:
        y_proba = None
        auc = np.nan

    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    print(f"Report model{report}")
    macro_p = report["macro avg"]["precision"]
    macro_r = report["macro avg"]["recall"]
    macro_f1 = report["macro avg"]["f1-score"]
    cm = confusion_matrix(y_test, y_pred)

    return {
        "acc": acc,
        "macro_p": macro_p,
        "macro_r": macro_r,
        "macro_f1": macro_f1,
        "macro_auc_ovr": auc,
        "cm": cm,
    }

# 2. Evaluate

In [13]:
def train_test(models, X_train, y_train, X_test, y_test):
    results = []
    trained = {}
    for name, model in models.items():
        pipe = Pipeline(steps=[
            ("preprocess", preprocess),
            ("clf", model),
        ])
        pipe.fit(X_train, y_train)
        
        
        metrics = evaluate(pipe, X_test, y_test)
        trained[name] = pipe

        results.append({
            "model": name,
            "acc": metrics["acc"],
            "macro_p": metrics["macro_p"],
            "macro_r": metrics["macro_r"],
            "macro_f1": metrics["macro_f1"],
            "macro_auc_ovr": metrics["macro_auc_ovr"],
        })
    return trained, results


### 2.1 Train and test

In [14]:
model_trained, results = train_test(models, X_train, y_train, X_test, y_test)
results_df = pd.DataFrame(results).sort_values(by="macro_f1", ascending=False)

Report model{'Insufficient_Weight': {'precision': 0.5909090909090909, 'recall': 0.7222222222222222, 'f1-score': 0.65, 'support': 54.0}, 'Normal_Weight': {'precision': 0.5476190476190477, 'recall': 0.39655172413793105, 'f1-score': 0.46, 'support': 58.0}, 'Obesity_Type_I': {'precision': 0.5425531914893617, 'recall': 0.7285714285714285, 'f1-score': 0.6219512195121951, 'support': 70.0}, 'Obesity_Type_II': {'precision': 0.5760869565217391, 'recall': 0.8833333333333333, 'f1-score': 0.6973684210526315, 'support': 60.0}, 'Obesity_Type_III': {'precision': 0.9014084507042254, 'recall': 0.9846153846153847, 'f1-score': 0.9411764705882353, 'support': 65.0}, 'Overweight_Level_I': {'precision': 0.6428571428571429, 'recall': 0.46551724137931033, 'f1-score': 0.54, 'support': 58.0}, 'Overweight_Level_II': {'precision': 0.4375, 'recall': 0.1206896551724138, 'f1-score': 0.1891891891891892, 'support': 58.0}, 'accuracy': 0.624113475177305, 'macro avg': {'precision': 0.6055619828715154, 'recall': 0.614500141

### Report

In [15]:
print(results_df)

                model       acc   macro_p   macro_r  macro_f1  macro_auc_ovr
2       random_forest  0.860520  0.866419  0.859510  0.861153       0.979760
3                 knn  0.770686  0.761803  0.766021  0.753713       0.922992
1       decision_tree  0.742317  0.743070  0.737645  0.737551       0.847304
0  logistic_regresion  0.624113  0.605562  0.614500  0.585669       0.885143


# Model Save Pipeline

In [16]:
import joblib
best_name = results_df.iloc[0]["model"]
best_pipe = model_trained[best_name]
joblib.dump(best_pipe, f"{best_name}_best_model.joblib")
print("Saved:", best_name, "-> best_model.joblib")

Saved: random_forest -> best_model.joblib
